Proses import library yang dibutuhkan

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import json
import time

<h1>A. PROSES EXTRACT DATA DARI WEB</h1>

Mempersiapkan list untuk menyimpan data 

In [3]:
nama_produk, harga , product_category, keterangan, gambar_product, link = [],[],[],[],[],[]

Menyiapkan fungsi untuk cleaning dan transform data

In [ ]:
def text_cleaner(text):
  '''
  fungsi ini digunakan untuk membersihkan text dari karakter atau angka sehingga output akan berupa text yang bersih.
  '''
  cleaned_text = ''.join([char for char in text if char.isalpha() or char.isspace()])
  return cleaned_text

def pengganti_kurung(list):
  '''
  Fungsi ini digunakan untuk mengubah kurung [] menjadi kurung {}
  '''
  hasil_set_string = "{" + ", ".join(map(str, list)) + "}"
  return hasil_set_string

Tahap pengambilan data dari web

In [ ]:
driver = webdriver.Chrome() #mempersiapkan driver yang digunakan
base_url = 'https://www.ikea.co.id' #link untuk base pada link product

#merupakan kumpulan link pencarian product di ikea yang akan diambil datanya
product_urls = ['https://www.ikea.co.id/in/produk/ruang-kerja-rumah?sort=SALES&page=',
                'https://www.ikea.co.id/in/produk/ruang-kerja-kantor?sort=SALES&page='
                'https://www.ikea.co.id/in/produk/kotak-penyimpanan-dan-penataan?sort=SALES&page='
                'https://www.ikea.co.id/in/produk/perlengkapan-ruang-kerja?sort=RECOMMENDED&page='
                'https://www.ikea.co.id/in/produk/peralatan-dapur?sort=SALES&page='
                ]

#tahapan looping terhadap link yang diinginkan sesuai list product_urls
i=0
for url in product_urls:
  i += 1
  print(f'Mulai mengambil data link ke - {i}')

  #looping untuk page dari setiap product yang ingin kita cari, saya batasi 3 page saja setiap kategori produk
  for page in range(3):
    alamat = url #menyalin alamat dari list product_urls
    driver.get(alamat+str(page+1))#mesukkan url kedalam webdriver sesuai dengan page
    time.sleep(5) #memberikan jeda waktu agar page source dapan terload sempurna


    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    #proses ekstraks data dari element html dengan algoritma mengambil semua container dari product yang berisi data.
    for container in soup.find_all('div',{'class':'col-6 col-md-4 col-lg-3 p-0 itemBlock'}):#looping untuk setiap countainer
      nama_element = container.find('h6', {'class':"display-7"})
      harga_element = container.find('div', {'class':"itemPrice-wrapper"}).find('span')
      info = container.find('div', {'class':"productInfo"}).find('input')
      keterangan_element = container.find('span', {'class':"itemFacts"})
      gambar = container.find('span', {'class':"productImg"}).find_all('img')
      link_element = container.find('div',{'class':"d-flex flex-row"}).find('a')


      #data gambar product lebih dari satu dan juga atribut dari gambar ada dua macam untuk setiap product.
      #Data gambar akan dikumpulkan dalam list sementara, dilakukan try and except untuk mengambil data menyesuaikan dengan attribut nya.
      images = []
      for img in gambar:
        try:
          pic = img['src']
          images.append(pic)
        except:
          pic = img['data-src']
          images.append(pic)
      
      #di dalam tag input yang berada dalam div dengan class productInfo, terdapat attribut value yang berisi kumpulan data product berbentuk dictionary
      value = info['value']
      #untuk mengakses data dalam dictionary tersebut perlu diconvert menjadi json
      value_dict = json.loads(value.replace('&quot;', '"'))

      #mengambil value category dalam json 
      category = value_dict['category']
      category_list = category.split('-')  # Memisahkan string berdasarkan '-'
      category_list = [text_cleaner(item) for item in category_list] #karena setiap text masih mengandung karakter dan angka maka perlu dibersihkan dengan fungsi text_cleaner


      #proses memasukkan data hasil scrap ke dalam list sesuai dengan data masing masing
      nama_produk.append(nama_element.text)
      harga.append(harga_element['data-price'])
      product_category.append(category_list)
      keterangan.append(keterangan_element.text)
      gambar_product.append(images)
      link.append(base_url+link_element['href'])
    print(f'Halaman {page+1} DONE !!!!!')



Pengecekan banyak data yang didapat

In [11]:
len(nama_produk), len(harga) , len(product_category), len(keterangan), len(gambar_product)

(570, 570, 570, 570, 570)

Mengubah bentuk penyimpanan dari product_category dan gambar_product yang berupa list dengan kurung [] menjadi list dengan kurung {}. Hal ini dilakukan dengan tujuan ketika data dieksport menjadi csv lalu setelahnya dimasukkan ke dalam postgresql, ketika ingin menormalisai data menjadi 3NF bisa menggunakan fungsi UNNEST

In [12]:
product_category = [pengganti_kurung(data) for data in product_category]
gambar_product = [pengganti_kurung(data) for data in gambar_product]

Membuat dataframe dari list data yang didapat

In [13]:
df_ikea_product = pd.DataFrame({
  'nama' : nama_produk,
  'keterangan' : keterangan,
  'harga' : harga,
  'category' : product_category,
  'gambar' : gambar_product,
  'link_product' : link
  }
)

<h1>B. PROSES TRANFORM DATA</h1>

Mengganti tipe data kolom pada harga menjadi int64 agar sesuai dengan data

In [14]:
df_ikea_product['harga']=df_ikea_product['harga'].astype('int64')

Destructure data keterangan karena dapat dipisahkan lagi sesuai jenis data

In [3]:
df_expand = df_ikea_product['keterangan'].str.rsplit(',').apply(pd.Series) #proses memisahkan menjadi list dengan dengan delimiter ','

#Proses memsakukan data ke kolom baru pada dataframe
df_ikea_product['product_type'] = df_expand[0]
df_ikea_product['color_and_material'] = df_expand[1]
df_ikea_product['size'] = df_expand[2]
df_ikea_product['extra'] = df_expand[3]

<h3>Kasus pertama hasil destructure terdapat beberapa baris yang memiliki data list sebanyak 4, sedangkan mayoritas baris memiliki 3 list. Setelah dianalisis hal ini terjadi karena kelebihan 1 data tersebut harusnya menjadi satu dalam product_type.</h3>

langkah pertama cari lokasi data yang memiliki list berjumlah 4.

In [4]:
target = df_ikea_product['extra'].notnull()#idenya mencari kolom ke 4 yaitu extra yang nilainya tidak null
df_ikea_product.loc[target].head()

,nama,keterangan,harga,category,gambar,link_product,product_type,color_and_material,size,extra
205,EILIF,"layar, berdiri bebas, abu-abu, 80x150 cm",1999000,"{' Sekat meja dan pemisah ruangan ', ' Ruang k...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/ruang-kerja-k...,layar,berdiri bebas,abu-abu,80x150 cm
206,EILIF,"layar, berdiri bebas, abu-abu/putih, 80x150 cm",2549000,"{' Sekat meja dan pemisah ruangan ', ' Ruang k...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/ruang-kerja-k...,layar,berdiri bebas,abu-abu/putih,80x150 cm
210,EILIF,"layar, berdiri bebas, abu-abu/hitam, 80x150 cm",2549000,"{' Sekat meja dan pemisah ruangan ', ' Ruang k...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/ruang-kerja-k...,layar,berdiri bebas,abu-abu/hitam,80x150 cm
323,DRAGAN,"kotak, set isi 3, bambu, 23x17x14 cm",299000,"{' Kotak dan keranjang', ' Pengaturan dan peny...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/kotak-dan-ker...,kotak,set isi 3,bambu,23x17x14 cm
371,FRAKTA,"kantong belanja, besar, biru, 55x37x35 cm/71 l",9900,"{' Aksesoris tas dan perjalanan ', ' Pengatura...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/pengaturan-pa...,kantong belanja,besar,biru,55x37x35 cm/71 l


Menggabungkan kolom product_type dengan color_and_material

In [7]:
#dari taget yang didapat di atas kita ambil baris datanya menggunakan fungsi loc, lalu kita gabungkan
df_ikea_product.loc[target,'product_type'] = df_ikea_product.loc[target,'product_type']+df_ikea_product.loc[target,'color_and_material']
df_ikea_product.loc[target].head()

,nama,keterangan,harga,category,gambar,link_product,product_type,color_and_material,size,extra
205,EILIF,"layar, berdiri bebas, abu-abu, 80x150 cm",1999000,"{' Sekat meja dan pemisah ruangan ', ' Ruang k...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/ruang-kerja-k...,layar berdiri bebas,berdiri bebas,abu-abu,80x150 cm
206,EILIF,"layar, berdiri bebas, abu-abu/putih, 80x150 cm",2549000,"{' Sekat meja dan pemisah ruangan ', ' Ruang k...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/ruang-kerja-k...,layar berdiri bebas,berdiri bebas,abu-abu/putih,80x150 cm
210,EILIF,"layar, berdiri bebas, abu-abu/hitam, 80x150 cm",2549000,"{' Sekat meja dan pemisah ruangan ', ' Ruang k...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/ruang-kerja-k...,layar berdiri bebas,berdiri bebas,abu-abu/hitam,80x150 cm
323,DRAGAN,"kotak, set isi 3, bambu, 23x17x14 cm",299000,"{' Kotak dan keranjang', ' Pengaturan dan peny...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/kotak-dan-ker...,kotak set isi 3,set isi 3,bambu,23x17x14 cm
371,FRAKTA,"kantong belanja, besar, biru, 55x37x35 cm/71 l",9900,"{' Aksesoris tas dan perjalanan ', ' Pengatura...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/pengaturan-pa...,kantong belanja besar,besar,biru,55x37x35 cm/71 l


Selanjutnya menggeser semua values setiap kolom mulai dari color_and_material sampai extra ke kiri

In [8]:
df_ikea_product.loc[target,'color_and_material'] = df_ikea_product.loc[target,'size']
df_ikea_product.loc[target,'size'] = df_ikea_product.loc[target,'extra']
df_ikea_product.loc[target].head()

,nama,keterangan,harga,category,gambar,link_product,product_type,color_and_material,size,extra
205,EILIF,"layar, berdiri bebas, abu-abu, 80x150 cm",1999000,"{' Sekat meja dan pemisah ruangan ', ' Ruang k...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/ruang-kerja-k...,layar berdiri bebas,abu-abu,80x150 cm,80x150 cm
206,EILIF,"layar, berdiri bebas, abu-abu/putih, 80x150 cm",2549000,"{' Sekat meja dan pemisah ruangan ', ' Ruang k...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/ruang-kerja-k...,layar berdiri bebas,abu-abu/putih,80x150 cm,80x150 cm
210,EILIF,"layar, berdiri bebas, abu-abu/hitam, 80x150 cm",2549000,"{' Sekat meja dan pemisah ruangan ', ' Ruang k...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/ruang-kerja-k...,layar berdiri bebas,abu-abu/hitam,80x150 cm,80x150 cm
323,DRAGAN,"kotak, set isi 3, bambu, 23x17x14 cm",299000,"{' Kotak dan keranjang', ' Pengaturan dan peny...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/kotak-dan-ker...,kotak set isi 3,bambu,23x17x14 cm,23x17x14 cm
371,FRAKTA,"kantong belanja, besar, biru, 55x37x35 cm/71 l",9900,"{' Aksesoris tas dan perjalanan ', ' Pengatura...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/pengaturan-pa...,kantong belanja besar,biru,55x37x35 cm/71 l,55x37x35 cm/71 l


Problem teratasi sehingga kolom extra tidak digunakan lagi sehingga bisa didelete

In [9]:
df_ikea_product.drop('extra', axis=1, inplace=True)

<h3>Problem ke dua terdapat beberapa baris dimana value kolom color_and_material tertukar dengan value kolom size</h3>

Langkah pertama cari lokasi target yang datanya tertukar

In [10]:
#Idenya kita cari pada kolom color_and_material yang valuenya mengandung angka
target2 = df_ikea_product['color_and_material'].str.contains(r'\d', na=False)
df_ikea_product.loc[target2].head()

,nama,keterangan,harga,category,gambar,link_product,product_type,color_and_material,size
240,SKUBB,"kotak, set isi 6, putih",89900,"{' Kotak dan keranjang', ' Pengaturan dan peny...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/kotak-dan-ker...,kotak,set isi 6,putih
248,SKUBB,"kotak, set isi 6, abu-abu tua",89900,"{' Kotak dan keranjang', ' Pengaturan dan peny...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/kotak-dan-ker...,kotak,set isi 6,abu-abu tua
262,LJUNGAN,"keranjang, 25x20x20 cm",299000,"{' Kotak dan keranjang', ' Pengaturan dan peny...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/kotak-dan-ker...,keranjang,25x20x20 cm,NaN
269,LJUNGAN,"keranjang dg penutup, 20x10 cm",199000,"{' Kotak dan keranjang', ' Pengaturan dan peny...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/kotak-dan-ker...,keranjang dg penutup,20x10 cm,NaN
270,BROGRUND,"kotak, set isi 3, abu-abu transparan/putih",129000,"{' Kotak dan keranjang', ' Pengaturan dan peny...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/kotak-dan-ker...,kotak,set isi 3,abu-abu transparan/putih


Proses swap values dari target

In [14]:
#Kita ambil kolom dari lokasi target yang ingin kita swap. Karena kita ingin swap valuenya maka perlu diberikan .values karena data harus berbentuk array untuk dilakukan swap.
df_ikea_product.loc[target2, ['color_and_material','size']] = df_ikea_product.loc[target2, ['size','color_and_material']].values
df_ikea_product.loc[target2].head()

,nama,keterangan,harga,category,gambar,link_product,product_type,color_and_material,size
240,SKUBB,"kotak, set isi 6, putih",89900,"{' Kotak dan keranjang', ' Pengaturan dan peny...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/kotak-dan-ker...,kotak,putih,set isi 6
248,SKUBB,"kotak, set isi 6, abu-abu tua",89900,"{' Kotak dan keranjang', ' Pengaturan dan peny...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/kotak-dan-ker...,kotak,abu-abu tua,set isi 6
262,LJUNGAN,"keranjang, 25x20x20 cm",299000,"{' Kotak dan keranjang', ' Pengaturan dan peny...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/kotak-dan-ker...,keranjang,NaN,25x20x20 cm
269,LJUNGAN,"keranjang dg penutup, 20x10 cm",199000,"{' Kotak dan keranjang', ' Pengaturan dan peny...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/kotak-dan-ker...,keranjang dg penutup,NaN,20x10 cm
270,BROGRUND,"kotak, set isi 3, abu-abu transparan/putih",129000,"{' Kotak dan keranjang', ' Pengaturan dan peny...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/kotak-dan-ker...,kotak,abu-abu transparan/putih,set isi 3


karena keterangan sudah berhasil didestructure maka kolom keterangan dapat dihapus 

In [15]:
df_ikea_product.drop('keterangan', axis =1 , inplace=True)
df_ikea_product.head(10)

,nama,harga,category,gambar,link_product,product_type,color_and_material,size
0,ALEX,1350000,"{' Ruang kerja rumah', ' Ruang kerja ', 'Showr...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/ruang-kerja-r...,unit laci,putih,36x70 cm
1,LENNART,249000,"{' Ruang kerja rumah', ' Ruang kerja ', 'Showr...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/ruang-kerja-r...,unit laci,putih,NaN
2,MICKE,1799000,"{' Ruang kerja rumah', ' Meja kerja rumah ', '...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/ruang-kerja-r...,meja,putih,105x50 cm
3,HELMER,499000,"{' Ruang kerja rumah', ' Ruang kerja ', 'Showr...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/ruang-kerja-r...,unit laci beroda,putih,28x69 cm
4,MICKE,1999000,"{' Ruang kerja rumah', ' Meja kerja rumah ', '...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/ruang-kerja-r...,meja,putih,142x50 cm
5,LENNART,249000,"{' Ruang kerja rumah', ' Ruang kerja ', 'Showr...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/ruang-kerja-r...,unit laci,hitam,NaN
6,ADILS/LINNMON,479000,"{' Ruang kerja rumah', ' Meja kerja rumah ', '...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/ruang-kerja-r...,meja,putih,100x60 cm
7,LAGKAPTEN/ADILS,679000,"{' Ruang kerja rumah', ' Meja kerja rumah ', '...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/ruang-kerja-r...,meja,putih,120x60 cm
8,LAGKAPTEN/ADILS,1249000,"{' Ruang kerja rumah', ' Meja kerja rumah ', '...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/ruang-kerja-r...,meja,putih,200x60 cm
9,HELMER,599000,"{' Ruang kerja rumah', ' Ruang kerja ', 'Showr...",{'https://d2xjmi1k71iy2m.cloudfront.net/dairyf...,https://www.ikea.co.id/in/produk/ruang-kerja-r...,unit laci beroda,hitam,28x69 cm


Menampilkan struktur dari dataframe

In [16]:
df_ikea_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   nama                570 non-null    object
 1   harga               570 non-null    int64 
 2   category            570 non-null    object
 3   gambar              570 non-null    object
 4   link_product        570 non-null    object
 5   product_type        570 non-null    object
 6   color_and_material  546 non-null    object
 7   size                387 non-null    object
dtypes: int64(1), object(7)
memory usage: 35.8+ KB


Export data menjadi csv untuk dimasukkan dalam database selanjutnya

In [17]:
df_ikea_product.to_csv('product_ikea.csv', index= False)